In [ ]:
import numpy as np
import pandas as pd
import re
import string
import nltk
from nltk.corpus import stopwords
from fuzzywuzzy import process, fuzz

In [ ]:
dataset=pd.read_csv('../input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv')
citiesdb=pd.read_csv('../input/pakistan-cities-and-postal-codes/Pakistan Cities and Zip Codes.csv')

In [ ]:
dataset.head()

In [ ]:
dataset.City.value_counts()

In [ ]:
dataset.City=dataset.City.astype('str')

In [ ]:
# lowering the text
dataset.City=dataset.City.apply(lambda x:x.lower() )

dataset.City.head()

In [ ]:
dataset.City=dataset.City.str.split(':',expand=True)
dataset.City=dataset.City.str.split(',',expand=True)
dataset.City=dataset.City.str.split('/',expand=True)

In [ ]:
dataset.City.value_counts()

In [ ]:
dataset.City

In [ ]:
token=nltk.tokenize.RegexpTokenizer(r'\w+')
#applying token
dataset.City=dataset.City.apply(lambda x:token.tokenize(x))

#view
display(dataset.City.head())

In [ ]:
#removing stop words
dataset.City=dataset.City.apply(lambda x:[w for w in x if w not in stopwords.words('english')])

#view
dataset.City.head()

In [ ]:
dataset.City=dataset.City.apply(lambda x:" ".join(x))
#View
dataset.City.head()

In [ ]:
dataset.City.value_counts()

In [ ]:
unique_city = dataset['City'].unique().tolist()
unique_city[4:10] 

In [ ]:
process.extract('islamabad', unique_city, scorer=fuzz.token_sort_ratio)

In [ ]:
process.extract('rajanpur', unique_city, scorer=fuzz.token_sort_ratio)

In [ ]:
score_sort = [(x,) + i
             for x in citiesdb.Area_Name 
             for i in process.extract(x, unique_city,     scorer=fuzz.token_sort_ratio)]

In [ ]:
similarity_sort = pd.DataFrame(score_sort, columns=['City_sort','match_sort','score_sort'])
similarity_sort.head()

In [ ]:
similarity_sort['sorted_City_sort'] = np.minimum(similarity_sort['City_sort'], similarity_sort['match_sort'])
similarity_sort.head()

In [ ]:
high_score_sort = similarity_sort[(similarity_sort['score_sort'] >= 95) &
                (similarity_sort['City_sort'] !=  similarity_sort['match_sort']) &
                (similarity_sort['sorted_City_sort'] != similarity_sort['match_sort'])]
high_score_sort = high_score_sort.drop('sorted_City_sort',axis=1).copy()

In [ ]:
high_score_sort

In [ ]:
high_score_sort.groupby(['City_sort','score_sort']).agg(
                        {'match_sort': ', '.join}).sort_values(
                        ['score_sort'], ascending=False)

In [ ]:
for x in range(len(high_score_sort.City_sort)):
    dataset.City= dataset.City.replace([high_score_sort.match_sort.iloc[x]],[high_score_sort.City_sort.iloc[x]])

In [ ]:
dataset.City=dataset.City.str.title()

In [ ]:
dataset.City.value_counts()

In [ ]:
dataset.head()

In [ ]:
original=pd.read_csv('../input/gufhtugu-publications-dataset-challenge/GP Orders - 5.csv')

In [ ]:
original['Diff']=(original['City'].str.len())-(dataset['City']).str.len()

In [ ]:
original.loc[original['Diff'] != 0, 'Changed?'] = True 
original.loc[original['Diff'] == 0, 'Changed?'] = False

In [ ]:
print(original['Changed?'].sum(),"Changes were made")

In [ ]:
dataset.to_csv('clean_city.csv')

# I have improved 🤗 the data but there is alot more work to be done to get only names. In this Notebook I have use fuzzywuzzy and nltk for extracting and then improving overall cities name. If you like my work do Upvote 👆